MILESTONE 1

In [1]:
# STEP 1: Import libraries
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Change path to your CSV location
file_path = '/content/drive/MyDrive/GlobalWeatherRepository.csv'
df = pd.read_csv(file_path, low_memory=False)

In [4]:
# STEP 3: Basic info
print("Shape:", df.shape)
print("\nData Types:\n", df.dtypes)
print("\nFirst 5 rows:\n", df.head())

Shape: (88468, 41)

Data Types:
 country                          object
location_name                    object
latitude                        float64
longitude                       float64
timezone                         object
last_updated_epoch                int64
last_updated                     object
temperature_celsius             float64
temperature_fahrenheit          float64
condition_text                   object
wind_mph                        float64
wind_kph                        float64
wind_degree                       int64
wind_direction                   object
pressure_mb                     float64
pressure_in                     float64
precip_mm                       float64
precip_in                       float64
humidity                          int64
cloud                             int64
feels_like_celsius              float64
feels_like_fahrenheit           float64
visibility_km                   float64
visibility_miles                float64
uv_inde

In [5]:
# STEP 4: Check for missing values
missing_summary = df.isnull().sum().reset_index()
missing_summary.columns = ['Column', 'Missing_Values']
missing_summary['Missing_%'] = (missing_summary['Missing_Values'] / len(df)) * 100
print("\nMissing Values Summary:\n", missing_summary.sort_values("Missing_%", ascending=False))


Missing Values Summary:
                           Column  Missing_Values  Missing_%
0                        country               0        0.0
1                  location_name               0        0.0
2                       latitude               0        0.0
3                      longitude               0        0.0
4                       timezone               0        0.0
5             last_updated_epoch               0        0.0
6                   last_updated               0        0.0
7            temperature_celsius               0        0.0
8         temperature_fahrenheit               0        0.0
9                 condition_text               0        0.0
10                      wind_mph               0        0.0
11                      wind_kph               0        0.0
12                   wind_degree               0        0.0
13                wind_direction               0        0.0
14                   pressure_mb               0        0.0
15            

In [6]:
# STEP 5: Inspect numerical columns (stats)
print("\nNumerical Summary:\n", df.describe())


Numerical Summary:
            latitude     longitude  last_updated_epoch  temperature_celsius  \
count  88468.000000  88468.000000        8.846800e+04         88468.000000   
mean      19.125807     22.172462        1.735550e+09            22.688292   
std       24.470764     65.836842        1.137876e+07             9.087760   
min      -41.300000   -175.200000        1.715849e+09           -24.900000   
25%        3.750000     -6.836100        1.725798e+09            18.000000   
50%       17.250000     23.316700        1.735555e+09            25.000000   
75%       40.400000     50.580000        1.745399e+09            28.400000   
max       64.150000    179.220000        1.755160e+09            49.200000   

       temperature_fahrenheit      wind_mph      wind_kph   wind_degree  \
count            88468.000000  88468.000000  88468.000000  88468.000000   
mean                72.840627      8.290632     13.345725    170.576988   
std                 16.357748      8.131214     13.

In [7]:
#print the column names
print(df.columns.tolist())

['country', 'location_name', 'latitude', 'longitude', 'timezone', 'last_updated_epoch', 'last_updated', 'temperature_celsius', 'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph', 'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in', 'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius', 'feels_like_fahrenheit', 'visibility_km', 'visibility_miles', 'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide', 'air_quality_Ozone', 'air_quality_Nitrogen_dioxide', 'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10', 'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination']


In [8]:
#drop the similar columns
df = df.drop(columns=['temperature_fahrenheit']) # Already dropped
df = df.drop(columns=['feels_like_fahrenheit']) # Dropping Celsius in a new cell

In [9]:
#print the column names to check whether the similar columns are dropped or not
print(df.columns.tolist())

['country', 'location_name', 'latitude', 'longitude', 'timezone', 'last_updated_epoch', 'last_updated', 'temperature_celsius', 'condition_text', 'wind_mph', 'wind_kph', 'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in', 'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km', 'visibility_miles', 'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide', 'air_quality_Ozone', 'air_quality_Nitrogen_dioxide', 'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10', 'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination']


In [10]:
# STEP 7: Parse date column
# Replace 'date' with the actual date column name in your dataset
df['last_updated'] = pd.to_datetime(df['last_updated'], errors='coerce')

In [11]:
# STEP 8: Aggregate daily → monthly
# This reduces size and helps with seasonal trends
monthly_df = (
    df.groupby([pd.Grouper(key='last_updated', freq='ME'), 'country'])  # adjust 'country' column name if needed
    .agg({
        'temperature_celsius': 'mean',
        'humidity': 'mean',
        'precip_mm': 'sum',
        'wind_kph': 'mean'
    })
    .reset_index()
)

print("\nMonthly Aggregated Data:\n", monthly_df.head())


Monthly Aggregated Data:
   last_updated      country  temperature_celsius   humidity  precip_mm  \
0   2024-05-31  Afghanistan            20.305882  38.058824       1.85   
1   2024-05-31      Albania            25.647059  57.529412       5.02   
2   2024-05-31      Algeria            26.117647  45.705882       0.01   
3   2024-05-31      Andorra            12.423529  60.882353       5.98   
4   2024-05-31       Angola            29.088235  69.764706       0.00   

    wind_kph  
0  11.517647  
1  13.076471  
2  22.617647  
3   9.876471  
4  19.594118  


In [12]:
# STEP 9: Save cleaned & aggregated dataset
df.to_csv("cleaned_weather_data.csv", index=False)
monthly_df.to_csv("monthly_weather_data.csv", index=False)

print("\n✅ Data cleaning complete. Files saved: cleaned_weather_data.csv, monthly_weather_data.csv")


✅ Data cleaning complete. Files saved: cleaned_weather_data.csv, monthly_weather_data.csv
